In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
LEARNED_THR = 0.00001
IOU_THR = 0.7
PREDICTION_PATH = "dataset/predictions/08-predictions.csv"

In [3]:
PREDICTION_IMAGE_PATH = Path("dataset/images/test")

test_df = pd.read_csv(PREDICTION_PATH).rename(
    columns={"x_min": "xmin", "y_min": "ymin", "x_max": "xmax", "y_max": "ymax"}
)
test_df.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
90523,ID_wnuBs6.jpg,healthy,0.009163,0.000000,153.250092,13.601792,193.023315
135874,ID_Qe7Lg1.jpeg,healthy,0.001720,1726.634033,0.153779,3008.601562,524.187317
115967,ID_pIZoja.jpeg,cssvd,0.000094,3553.577148,1324.986328,4000.000000,1800.000000
84353,ID_QOVg3l.jpg,anthracnose,0.000014,0.000000,266.232971,99.120544,770.942017
38729,ID_RC0I1m.jpg,anthracnose,0.000024,353.334198,184.377548,810.000000,966.811279


In [4]:
class_id = {j: i for i, j in enumerate(sorted(test_df["class"].unique()))}
test_df["class_id"] = test_df["class"].map(class_id)
class_id

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [5]:
test_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,anthracnose,0.002872,60.666199,15.254377,4000.000000,1679.711670,0
1,ID_cWEAQI.jpeg,anthracnose,0.000300,3672.035400,84.970467,4000.000000,579.548340,0
2,ID_cWEAQI.jpeg,anthracnose,0.000245,930.349670,444.719177,2732.435547,1620.193848,0
3,ID_cWEAQI.jpeg,anthracnose,0.000188,228.629547,0.000000,909.342712,437.895020,0
4,ID_cWEAQI.jpeg,anthracnose,0.000111,3156.461914,20.296572,3988.080322,1098.258911,0


In [6]:
import torch

def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}


converted_labels = list(PREDICTION_IMAGE_PATH.glob("*"))
converted_labels = [i.name for i in converted_labels]

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	print(len(preds))
	print(list(preds.keys())[:10])
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [7]:
test_df.sample(2)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
84969,ID_h8uMuy.jpg,anthracnose,0.000045,0.000000,373.052246,110.937576,611.452881,0
112809,ID_U0BYn7.jpg,cssvd,0.000013,1736.422729,0.000000,2047.461548,389.568176,1


In [8]:
predictions = get_preds_data(test_df, thr=None)

len(predictions)

1626
['ID_A16nzu.jpg', 'ID_A1Euyz.jpg', 'ID_A1HcV0.jpeg', 'ID_A4ZdJC.jpeg', 'ID_A5SFUW.jpeg', 'ID_A6Fogi.jpeg', 'ID_ABDCyn.jpeg', 'ID_ACg6Qf.jpeg', 'ID_AFi8A1.jpg', 'ID_AHlc9P.jpg']


1626

In [9]:
# predictions["ID_CGnAYP.jpg"]

In [10]:
predictions["ID_CGnAYP.jpg"].keys()

dict_keys(['boxes', 'scores', 'labels'])

In [11]:
from torchvision.ops import nms

def apply_nms(predictions: dict[str, torch.Tensor], iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression (NMS) to reduce overlapping boxes.

    Args:
        predictions (dict): Dictionary containing keys ['boxes', 'scores', 'labels'].
        iou_threshold (float): Intersection over Union (IoU) threshold for NMS.

    Returns:
        dict: Filtered predictions after applying NMS.
    """
    filtered_predictions = {}
    for image_id, data in predictions.items():
        boxes = data["boxes"]
        scores = data["scores"]
        labels = data["labels"]

        if boxes.numel() == 0:
            # If no boxes, skip this image
            filtered_predictions[image_id] = data
            continue

        # Perform NMS for each class separately
        keep_indices = []
        for label in labels.unique():
            label_mask = labels == label
            label_boxes = boxes[label_mask]
            label_scores = scores[label_mask]
            indices = nms(label_boxes, label_scores, iou_threshold)
            keep_indices.extend(label_mask.nonzero(as_tuple=True)[0][indices].tolist())

        # Filter boxes, scores, and labels based on NMS results
        keep_indices = torch.tensor(keep_indices, dtype=torch.long)
        filtered_predictions[image_id] = {
            "boxes": boxes[keep_indices],
            "scores": scores[keep_indices],
            "labels": labels[keep_indices],
        }

    return filtered_predictions

# Apply NMS to predictions
filtered_predictions = apply_nms(predictions, iou_threshold=IOU_THR)

In [12]:
# for k, v in predictions.items():
#     r = filtered_predictions[k]
#     if len(v["boxes"]) != len(r["boxes"]):
#         print(k, len(v["boxes"]), len(r["boxes"]))
#         print("Boxes before NMS:", v["boxes"])
#         print("Boxes after NMS:", r["boxes"])
#         print("Scores before NMS:", v["scores"])
#         print("Scores after NMS:", r["scores"])
#         print("Labels before NMS:", v["labels"])
#         print("Labels after NMS:", r["labels"])
#         print()
#         break

In [13]:
filtered_data = []
id_to_label = {v: k for k, v in class_id.items()}
for image_id, data in filtered_predictions.items():
    boxes = data["boxes"].tolist()
    scores = data["scores"].tolist()
    labels = data["labels"].tolist()
    for box, score, label in zip(boxes, scores, labels):
        filtered_data.append({
            "Image_ID": image_id,
            "class": id_to_label[label],
            "confidence": score,
            "ymin": box[1],
            "xmin": box[0],
            "ymax": box[3],
            "xmax": box[2],
            "class_id": label
        })

filtered_df = pd.DataFrame(filtered_data)
filtered_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,anthracnose,0.002872,60.666199,15.254377,4000.000000,1679.711670,0
1,ID_cWEAQI.jpeg,anthracnose,0.000300,3672.035400,84.970467,4000.000000,579.548340,0
2,ID_cWEAQI.jpeg,anthracnose,0.000245,930.349670,444.719177,2732.435547,1620.193848,0
3,ID_cWEAQI.jpeg,anthracnose,0.000188,228.629547,0.000000,909.342712,437.895020,0
4,ID_cWEAQI.jpeg,anthracnose,0.000111,3156.461914,20.296572,3988.080322,1098.258911,0


In [14]:
len(filtered_df), len(test_df)

(146305, 146340)

In [15]:
filtered_df["class"].value_counts()

class
anthracnose    48775
healthy        48770
cssvd          48760
Name: count, dtype: int64

In [16]:
files = list(PREDICTION_IMAGE_PATH.glob("*"))

In [17]:
test_df: pd.DataFrame = filtered_df.copy()

In [18]:
test_records = test_df.to_dict(orient="records")

test_records[:5]

[{'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0028718397952616215,
  'ymin': 60.66619873046875,
  'xmin': 15.254377365112305,
  'ymax': 4000.0,
  'xmax': 1679.711669921875,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0003004346799571067,
  'ymin': 3672.035400390625,
  'xmin': 84.97046661376953,
  'ymax': 4000.0,
  'xmax': 579.54833984375,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.00024532550014555454,
  'ymin': 930.3496704101562,
  'xmin': 444.71917724609375,
  'ymax': 2732.435546875,
  'xmax': 1620.19384765625,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.00018821493722498417,
  'ymin': 228.62954711914062,
  'xmin': 0.0,
  'ymax': 909.3427124023438,
  'xmax': 437.89501953125,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.00011063480633310974,
  'ymin': 3156.46

In [19]:
file_id = [i.name for i in files]
file_id[:2]

['ID_cWEAQI.jpeg', 'ID_NtqErb.jpg']

In [20]:
file_okay = set(test_df["Image_ID"].values)
file_nokay = set(file_id) - file_okay

for f in file_nokay:
    test_records.append(
		{
			"Image_ID": f,
		}
	)

In [21]:
test_df = pd.DataFrame(test_records)

test_df.sample(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
70504,ID_wVucYf.jpg,cssvd,0.000145,578.748962,112.342033,985.176208,453.612640,1
70080,ID_XzSqhK.JPG,healthy,0.000126,2691.659668,317.179352,2984.715820,690.513000,2
106979,ID_Hp6I1X.jpg,healthy,0.000939,9.654806,260.260498,70.364113,340.545197,2
135292,ID_SxZRvt.jpg,cssvd,0.000110,457.339539,12.926865,1280.000000,302.216431,1
118254,ID_l7Rbyv.jpeg,anthracnose,0.000059,0.000000,2549.964355,376.218384,3019.970215,0
114753,ID_kFZBmo.jpg,anthracnose,0.000006,317.687714,1462.158569,660.097290,1763.766479,0
141812,ID_lDkNRk.jpeg,anthracnose,0.000676,489.822571,890.065002,1808.115234,1625.627319,0
26353,ID_d0gpda.jpg,healthy,0.000182,2731.615967,285.903015,3049.183350,503.163147,2
93899,ID_WYxe7D.jpeg,cssvd,0.000346,1385.986694,1105.578247,2170.759277,2165.531006,1
145903,ID_EWolCN.jpeg,cssvd,0.000043,3654.948975,724.952759,4032.000000,2135.825928,1


In [22]:
test_df["class"].value_counts()

class
anthracnose    48775
healthy        48770
cssvd          48760
Name: count, dtype: int64

In [23]:
test_df.to_csv(PREDICTION_PATH.replace("-prediction", "-submission"), index=False)

In [24]:
PREDICTION_PATH.replace("-prediction", "-submission")

'dataset/predictions/08-submissions.csv'